# Setup kaggle notebook

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Install GFootball Environment

make sure to set internet on

In [ ]:
# Install:
# Kaggle environments.
!git clone https://github.com/Kaggle/kaggle-environments.git
!cd kaggle-environments && pip install .

# GFootball environment.
!apt-get update -y
!apt-get install -y libsdl2-gfx-dev libsdl2-ttf-dev

# Make sure that the Branch in git clone and in wget call matches !!
!git clone -b v2.7 https://github.com/google-research/football.git
!mkdir -p football/third_party/gfootball_engine/lib

!wget https://storage.googleapis.com/gfootball/prebuilt_gameplayfootball_v2.7.so -O football/third_party/gfootball_engine/lib/prebuilt_gameplayfootball.so
!cd football && GFOOTBALL_USE_PREBUILT_SO=1 pip3 install .

#  **Install clingo**

In [ ]:
!conda install -y -c potassco clingo

# Play Agent - ASP

In [ ]:
%%writefile submission.py
from kaggle_environments.envs.football.helpers import *
import clingo
import re
import math

# AZIONI

# Action.Idle = 0
# Action.Left = 1, run to the left, sticky action.
# Action.TopLeft = 2, run to the top-left, sticky action.
# Action.Top = 3, run to the top, sticky action.
# Action.TopRight = 4, run to the top-right, sticky action.
# Action.Right = 5, run to the right, sticky action.
# Action.BottomRight = 6, run to the bottom-right, sticky action.
# Action.Bottom = 7, run to the bottom, sticky action.
# Action.BottomLeft = 8, run to the bottom-left, sticky action.
# Action.Sprint = 9, start sprinting, sticky action. Player moves faster, but has worse ball handling.
# Action.ReleaseSprint = 10, stop sprinting.
# Action.LongPass = 11, perform a long pass to the player on your team. Player to pass the ball to is auto-determined based on the movement direction.
# Action.HighPass = 12, perform a high pass, similar to action_long_pass.
# Action.ShortPass = 13, perform a short pass, similar to action_long_pass.
# Action.Shot = 14, perform a shot, always in the direction of the opponent's goal.
# Action.ReleaseDirection = 15, reset current movement direction.
# Action.Slide = 16, perform a slide 
# Action.Dribble = 17, start dribbling 
# Action.ReleaseDribble = 18, stop dribbling. 


# GAME MODES

# GameMode.Normal = 0, normal play
# GameMode.ThrowIn = 1, rimessa laterale
# GameMode.FreeKick = 2, calcio di punizione
# GameMode.Corner = 3, calcio d'angolo
# GameMode.Penalty = 4, calcio di rigore


def find_answer_in_model(string):
    start = 6
    end = 9
    index = string.rfind('answer')
    temp = string[index+start:index+end]
    answer = re.split("[()]+", temp)
    return(int(answer[1]))

def find_action_in_answer(x):
    _map = [
        Action.Idle,
        Action.Left,
        Action.TopLeft,
        Action.Top,
        Action.TopRight,
        Action.Right,
        Action.BottomRight,
        Action.Bottom,
        Action.BottomLeft,
        Action.Sprint,
        Action.ReleaseSprint,
        Action.LongPass,
        Action.HighPass,
        Action.ShortPass,
        Action.Shot,
        Action.ReleaseDirection,
        Action.Slide,
        Action.Dribble,
        Action.ReleaseDribble
    ]
    return _map[x]

def distanza(x1, y1, x2, y2):
    return math.sqrt((x1 - x2) ** 2 + (y1 * 2.38 - y2 * 2.38) ** 2) 

def get_active_sticky_actions(obs):
    active_sticky_actions = ''
    if (Action.Left in obs['sticky_actions']):
        active_sticky_actions += ('active_sticky_action(1). ')
    if (Action.TopLeft in obs['sticky_actions']):
        active_sticky_actions += ('active_sticky_action(2). ')
    if (Action.Top in obs['sticky_actions']):
        active_sticky_actions += ('active_sticky_action(3). ')
    if (Action.TopRight in obs['sticky_actions']):
        active_sticky_actions += ('active_sticky_action(4). ')
    if (Action.Right in obs['sticky_actions']):
        active_sticky_actions += ('active_sticky_action(5). ')
    if (Action.BottomRight in obs['sticky_actions']):
        active_sticky_actions += ('active_sticky_action(6). ')
    if (Action.Bottom in obs['sticky_actions']):
        active_sticky_actions += ('active_sticky_action(7). ')
    if (Action.BottomLeft in obs['sticky_actions']):
        active_sticky_actions += ('active_sticky_action(8). ')
    if (Action.Sprint in obs['sticky_actions']):
        active_sticky_actions += ('active_sticky_action(9). ')
    if (Action.Dribble in obs['sticky_actions']):
        active_sticky_actions += ('active_sticky_action(17). ')
    #print('ACTIVE STICKY ACTIONS: ' + active_sticky_actions)
    return active_sticky_actions

def get_active_player_position(obs):
    active_player_position = ''
    #print(str(int(obs['left_team'][obs['active']][1] * 1000)))
    if (obs['left_team'][obs['active']] != None and obs['left_team'][obs['active']][0] != None):
        active_player_position += 'active_player(' + str(obs['active']) + '). '
        active_player_position += 'active_player_x(' + str(int(obs['left_team'][obs['active']][0] * 1000)) + '). '
        active_player_position += 'active_player_y(' + str(int(obs['left_team'][obs['active']][1] * 1000)) + '). '
    #print('ACTIVE PLAYER: ' + active_player_position)
    return active_player_position

def get_ball_position(obs):
    ball_position = ''
    if (obs['ball'][0] != None and obs['ball'][1] != None):
        #print(str(int(obs['ball'][1] * 1000)))
        ball_position += 'ball_position_x(' + str(int(obs['ball'][0] * 1000)) + '). '
        ball_position += 'ball_position_y(' + str(int(obs['ball'][1] * 1000)) + '). '
    #print('BALL POSITION: ' + ball_position)
    return ball_position

def get_ball_direction(obs):
    ball_direction = ''
    if (obs['ball_direction'][0] != None and obs['ball_direction'][1] != None):
        #print(str(int(obs['ball_direction'][1] * 1000)))
        ball_direction += 'ball_direction_x(' + str(int(obs['ball_direction'][0] * 10000)) + '). '
        ball_direction += 'ball_direction_y(' + str(int(obs['ball_direction'][1] * 10000)) + '). '
    #print('BALL DIRECTION: ' + ball_direction)
    return ball_direction

def get_defense_vector(obs):
    defense_vector = ''
    player_x = obs['left_team'][obs['active']][0]
    player_y = obs['left_team'][obs['active']][1]
    ballPosX = obs['ball'][0]
    ballPosY = obs['ball'][1]
    vec_pgX = -1 - ballPosX
    vec_pgY = - ballPosY
    qX = -1 - vec_pgX + (vec_pgX * 0.2)
    qY = - vec_pgY + (vec_pgY * 0.2)
    vector_x = (qX - player_x)
    vector_y = (qY - player_y)
    defense_vector += 'defense_vector_x(' + str(int(vector_x * 1000)) + '). '
    defense_vector += 'defense_vector_y(' + str(int(vector_y * 1000)) + '). '
    return defense_vector

def get_ball_team(obs):
    ball_team = 'ball_team(' + str(obs['ball_owned_team']) + '). '
    #print('BALL TEAM: ' + ball_team)
    return ball_team

def get_ball_player(obs):
    ball_player = 'ball_player(' + str(obs['ball_owned_player']) + '). '
    #print('BALL PLAYER: ' + ball_player)
    return ball_player

def get_enemy_goalkeeper(obs):
    enemy_goalkeeper_x = 'enemy_goalkeeper_x(' + str(int(obs['right_team'][0][0] * 1000)) + '). '
    #print('BALL PLAYER: ' + ball_player)
    return enemy_goalkeeper_x

def get_game_mode(obs):
    if obs['game_mode'] == GameMode.Normal:
        return 'game_mode(0). '
    elif obs['game_mode'] == GameMode.ThrowIn:
        return 'game_mode(1). '
    elif obs['game_mode'] == GameMode.FreeKick:
        return 'game_mode(2). '
    elif obs['game_mode'] == GameMode.Corner:
        return 'game_mode(3). '
    elif obs['game_mode'] == GameMode.Penalty:
        return 'game_mode(4). '
    else:
        return 'game_mode(0).'
    
def get_closest_opponent_ahead(obs):
    posX = obs['left_team'][obs['active']][0]
    posY = obs['left_team'][obs['active']][1]
    shortest_distance = None
    opponent = None
    for i in range(1, len(obs["right_team"])):
        distance_to_opponent = distanza(posX, posY, obs["right_team"][i][0], obs["right_team"][i][1])
        if (obs["right_team"][i][0] > posX and
                abs(obs["right_team"][i][0] - posX) < 0.15 and
                abs(obs["right_team"][i][1] - posY) < 0.1):
            if shortest_distance == None or distance_to_opponent < shortest_distance:
                shortest_distance = distance_to_opponent
                opponent = obs["right_team"][i]
    opponent_ahead = ''
    if opponent != None:
        opponent_ahead += 'opponent_ahead(1). '
        opponent_ahead += 'opponent_ahead_x(' + str(int(opponent[0] * 1000)) + '). '
        opponent_ahead += 'opponent_ahead_y(' + str(int(opponent[1] * 1000)) + '). '
    else:
        opponent_ahead += 'opponent_ahead(0). '
    return opponent_ahead
    
def get_asp_program(obs):
    return """\
    
    #show.
    #show X : answer(X).
    
    
    
    % INPUTS
    
    % modalità di gioco : 0 = normal, 1 = rimessa laterale, 2 = punizione, 3 = angolo, 4 = rigore
    """ + get_game_mode(obs) + """
    
    % squadra in possesso della palla : 0 = giocatore, 1 = avversario, -1 = nessuno
    """ + get_ball_team(obs) + """
    
    % giocatore in possesso della palla : 0..11
    """ + get_ball_player(obs) + """
    
    % posizione del portiere avversario
    """ + get_enemy_goalkeeper(obs) + """
    
    % azioni in corso : 0..8 = direzione di movimento, 9 = corsa
    """ + get_active_sticky_actions(obs) + """
    
    % posizione del giocatore controllato
    """ + get_active_player_position(obs) + """
    
    % posizione della palla
    """ + get_ball_position(obs) + """
    
    % direzione della palla
    """ + get_ball_direction(obs) + """
    
    % posizione ottimale di copertura tra la palla e la nostra porta
    """ + get_defense_vector(obs) + """
    
    % avversario d'avanti più vicino
    """ + get_closest_opponent_ahead(obs) + """
    
    
    
    % ATTACCO E DIFESA
    
    % CORRI SE NON STAI CORRENDO
    action(9) :- not active_sticky_action(9), ball_team(K), K != 0.
    action(9) :- not active_sticky_action(9), ball_team(0), active_player_x(X), enemy_goalkeeper_x(X1), X < 300, X > -400, X1 - X > 350.
    
    % SMETTI DI CORRERE SE STAI PER TIRARE
    action(10) :- active_sticky_action(9), ball_team(0), active_player_x(X), enemy_goalkeeper_x(X1), X1 - X <= 350.
    
    
    
    % CALCI PIAZZATI
    
    % calcio di rigore
     % mira
    action(4) :- game_mode(4), ball_team(0), not active_sticky_action(4).
     % tira
    action(14) :- game_mode(4), ball_team(0), active_sticky_action(4).
    
    % calcio d'angolo
    action(12) :- game_mode(3), ball_team(0).
    
    % calcio di punizione
     % mira
    action(5) :- game_mode(2), ball_team(0), not active_sticky_action(5).
     % passa se sei non sei in buona posizione per tirare
    action(12) :- game_mode(2), ball_team(0).
     % tira se sei in buona posizione per tirare
    action(14) :- game_mode(2), ball_team(0), active_player_x(X), active_player_y(Y), Y > -160, Y < 160, X > 600.
    
    
    
    % ATTACCO
    
    % condizione di partenza : il giocatore controllato ha la palla
    player_has_ball :- game_mode(0), ball_team(0), active_player(K), ball_player(K).
    
    % (utility) calcola se la palla è davanti al giocatore controllato
    ball_is_ahead :- active_player_x(X), ball_position_x(X1), X1 > X.
    
    % rinvio portiere
    action(14) :- player_has_ball, active_player(0).
    
    % spazzata dalla difesa
    action(14) :- player_has_ball, active_player_x(X), X > -1000, X <= -800.
    
    % lancio lungo dalla difesa
    action(12) :- player_has_ball, run_to_goal(_), ball_is_ahead, active_player_x(X), X > -800, X <= -500.
    
    % passaggio filtrante
    action(11) :- player_has_ball, run_to_goal(_), ball_is_ahead, active_player_x(X), X > -500, X < -200.
    
    % tiro con portiere fuori posizione
     % (utility) check se il portiere è uscito dai pali
    goalkeeper_out :- player_has_ball, active_player_x(X), enemy_goalkeeper_x(X1), X1 <= 900, X1 - X < 350.
     % spiazzamento a sinistra
    action(6) :- goalkeeper_out, not active_sticky_action(6), active_player_y(Y), Y >= 0, Y < 160.
     % spiazzamento a destra
    action(4) :- goalkeeper_out, not active_sticky_action(4), active_player_y(Y), Y < 0, Y > -160.
     % tiro 
    action(14) :- goalkeeper_out, active_sticky_action(4).
    action(14) :- goalkeeper_out, active_sticky_action(6).
    
    % tiro in posizione avanzata
    action(14) :- player_has_ball, ball_is_ahead, active_player_x(X), active_player_y(Y), X > 700, Y > -160, Y < 160.
    
    % passaggio in posizione avanzata
    action(13) :- player_has_ball, not ball_is_ahead, active_player_x(X), active_player_y(Y), X > 700, Y > -160, Y < 160.
    
    % cross in posizione avanzata
    cross(3) :- player_has_ball, active_player_x(X), active_player_y(Y), X > 700, Y >= 160.
    cross(7) :- player_has_ball, active_player_x(X), active_player_y(Y), X > 700, Y <= -160.
    action(X) :- cross(X), not active_sticky_action(X).
    action(13) :- cross(X), active_sticky_action(X).
    
    % dribbling
    can_dribble(1) :- player_has_ball, opponent_ahead(1), active_player_y(Y), active_player_x(X), X > 0, Y > -300, Y < 300.
    %action(17) :- can_dribble(0). 
    %can_dribble(1) :- active_sticky_action(17).
    
    dribble(4) :- can_dribble(1), opponent_ahead_y(Y1), active_player_y(Y), Y < 0, Y1 > Y + 100.
    dribble(6) :- can_dribble(1), opponent_ahead_y(Y1), active_player_y(Y), Y < 0, Y1 < Y + 100, Y1 > Y - 100.
    dribble(5) :- can_dribble(1), opponent_ahead_y(Y1), active_player_y(Y), Y < 0, Y1 < Y - 100.
    
    dribble(5) :- can_dribble(1), opponent_ahead_y(Y1), active_player_y(Y), Y > 0, Y1 > Y + 100.
    dribble(6) :- can_dribble(1), opponent_ahead_y(Y1), active_player_y(Y), Y > 0, Y1 < Y + 100, Y1 > Y - 100.
    dribble(4) :- can_dribble(1), opponent_ahead_y(Y1), active_player_y(Y), Y > 0, Y1 < Y - 100.
    
    action(X) :- dribble(X).
    
    % rilascio dribbling
    action(18) :- active_sticky_action(17), opponent_ahead(0).
    
    % corri verso la porta avversaria
    run_to_goal(5) :- player_has_ball, active_player_y(Y), Y < 160, Y > -160.
    run_to_goal(4) :- player_has_ball, active_player_y(Y), Y > 160.
    run_to_goal(6) :- player_has_ball, active_player_y(Y), Y < -160.
    action(X) :- run_to_goal(X).
    
    
    
    % DIFESA
    
    % condizione di partenza : la palla non è in nostro possesso
    defense_move(X) :- ball_team(X), X != 0.
    
    % tattica 1: corri verso la palla se sei vicino ad essa
    defense(1) :- defense_move(1), active_player_x(X), active_player_y(Y), ball_position_x(X1), ball_position(Y1), X < X1, X > X1 - 50, Y1 - Y < 20, Y1 - Y > -20.
    
    % tattica 2: corri verso la direzione in cui sta andando la palla se sei nella metà campo avversaria
    defense(2) :- defense_move(1), active_player_x(X), X > 0.
    defense(2) :- defense_move(-1).
    
    % tattica 3: copri la porta
    defense(3) :- defense_move(1).
    
    %not run_to_ball_position(0), not run_to_ball_direction(0).
    
    % calcolo tattica 1
    x_dir(X, 1) :- defense(1), ball_position_x(Y), active_player_x(Z), X = Z - Y. 
    y_dir(X, 1) :- defense(1), ball_position_y(Y), active_player_y(Z), X = Z - Y. 
    
    % calcolo tattica 2
    x_dir(X, 2) :- defense(2), ball_position_x(Y), ball_direction_x(Y1), active_player_x(Z), X = Y + Y1 - Z.
    y_dir(X, 2) :- defense(2), ball_position_y(Y), ball_direction_y(Y1), active_player_y(Z), X = Y + Y1 - Z.
    
    % calcolo tattica 3
    x_dir(X, 3) :- defense(3), defense_vector_x(X).
    y_dir(X, 3) :- defense(3), defense_vector_y(X).
    
    % applica tattica difensiva scelta : calcola direzione dove andare 1
    dirsign_x(0, K) :- x_dir(X, K), X < -20.
    dirsign_x(1, K) :- x_dir(X, K), X <= 20, X >= -20.
    dirsign_x(2, K) :- x_dir(X, K), X > 20.
    dirsign_y(0, K) :- y_dir(X, K), X < -20.
    dirsign_y(1, K) :- y_dir(X, K), X <= 20, X >= -20.
    dirsign_y(2, K) :- y_dir(X, K), X > 20.
    
    % applica tattica difensiva scelta : calcola direzione dove andare 2
     % top left
    direction(2, K) :- dirsign_y(0, K), dirsign_x(0, K).
     % top
    direction(3, K) :- dirsign_y(0, K), dirsign_x(1, K).
     % top right
    direction(4, K) :- dirsign_y(0, K), dirsign_x(2, K).
     % left
    direction(1, K) :- dirsign_y(1, K), dirsign_x(0, K).
     % release direction
    direction(15, K) :- dirsign_y(1, K), dirsign_x(1, K).
     % right
    direction(5, K) :- dirsign_y(1, K), dirsign_x(2, K).
     % bottom left
    direction(8, K) :- dirsign_y(2, K), dirsign_x(0, K).
     % bottom
    direction(7, K) :- dirsign_y(2, K), dirsign_x(1, K).
     % bottom right
    direction(6, K) :- dirsign_y(2, K), dirsign_x(2, K).
    
    % esegui azione difensiva
    action(X) :- direction(X, K).
    
    
    
    % FILLER
    
    % azione filler quando non ci sono altre azioni disponibili
    action(0).
    
    
    
    % MAIN
    
    % guess and check answer
    {act(X)} :- action(X).
    answer(X) :- act(X).
    
    % azione scelta deve essere unica
    :- #count{X: act(X)} != 1.
    
    
    
    % OTTIMIZZAZIONE
    
    % azione nulla : priorità minima
    :~ act(0). [25@5, act(0)]
    
    % corri verso la porta avversaria : priorità minore rispetto alle altre azioni d'attacco
    :~ run_to_goal(X), act(X). [1@1, act(X)]
    
    % tattica difensiva 3 : priorità bassa
    :~ act(X), direction(X, 3). [1@2, direction(X, 3)]
    
    % dribbling : priorità media
    :~ not act(X), dribble(X). [1@2, dribble(X)]
    
    % tiro : priorità alta
    :~ not act(14), action(14). [1@3, act(14)]
    
    % azione corsa o rilascio corsa : priorità massima
    :~ not act(9), action(9). [1@4, act(9)]
    :~ not act(10), action(10). [1@4, act(10)]
    
    
    """

@human_readable_agent
def agent(obs):
    
    #inizializziamo il processo di grounding/solving
    ctl = clingo.Control()
    
    #ASP
    ctl.add("base", [], get_asp_program(obs))
    
    ctl.ground([("base", [])])
    ctl.configuration.solve.models="0"
    
    models = []
    
    # little magic
    with ctl.solve(yield_=True) as handle:
         for model in handle:
             models.append(model.symbols(atoms=True))
    
    #print(str(models[0]))
    answer = find_answer_in_model(str(models))
    action = find_action_in_answer(answer)
    return action


 # Custom Opponent
 very strong

In [ ]:
%%writefile agent.py
from kaggle_environments.envs.football.helpers import *
from math import sqrt

directions = [
             [Action.TopLeft,      Action.Top,     Action.TopRight],
             [Action.Left,        Action.Idle,        Action.Right],
             [Action.BottomLeft, Action.Bottom, Action.BottomRight]
             ]

def dirsign(x):
    #se si trova molto vicino all'obiettivo
    if abs(x) < 0.02:
        return 1
    #se si trova a destra dell'obiettivo
    elif x < 0:
        return 0
    #se si trova a sinistra dell'obiettivo
    else:
        return 2
    
def pallaAvanti(pos, ballPos):
    return pos < ballPos

#strategia difensiva
def defensiveMove(obs):
    posX = obs['left_team'][obs['active']][0]
    posY = obs['left_team'][obs['active']][1]
    ballPosX = obs['ball'][0]
    ballPosY = obs['ball'][1]
    ballDirX = obs['ball_direction'][0]
    ballDirY = obs['ball_direction'][1]
    if posX < ballPosX and abs(posX - ballPosX) < 0.05 and abs(posY - ballPosY) < 0.02:
        return runTowardsTheBall(posX, posY, ballPosX, ballPosY)
    elif posX > 0:
        return runTowardsTheBallDir(posX, posY, ballPosX, ballPosY, ballDirX, ballDirY)
    else:
        return copertura(posX, posY, ballPosX, ballPosY)
    
#tackle
def runTowardsTheBall(posX, posY, ballPosX, ballPosY):
    xdir = dirsign(ballPosX - posX)
    ydir = dirsign(ballPosY - posY)
    return directions[ydir][xdir] 
    
#anticipo
def runTowardsTheBallDir(posX, posY, ballPosX, ballPosY, ballDirX, ballDirY):
    xdir = dirsign(ballPosX + (ballDirX * 12) - posX)
    ydir = dirsign(ballPosY + (ballDirY * 12) - posY)
    #print('ballDirX: ' + str(ballDirX))
    #print('ballDirY: ' + str(ballDirY))
    return directions[ydir][xdir]

#copertura
def copertura(posX, posY, ballPosX, ballPosY):
    vec_pgX = -1 - ballPosX
    vec_pgY = - ballPosY
    qX = -1 - vec_pgX + (vec_pgX * 0.2)
    qY = - vec_pgY + (vec_pgY * 0.2)
    xdir = dirsign(qX - posX)
    ydir = dirsign(qY - posY)
    return directions[ydir][xdir] 

#calcola il giocatore piu avanzato della squadra
def mostAdvancedPlayer(obs):
    for i in range(11):
        flag = True
        for j in range(11):
            if obs['left_team'][i][0] < obs['left_team'][j][0]:
                flag = False
        if flag == True:
            return i
    return None

#verifica se il giocatore più avanzato della squadra è in fuorigioco
def atkInOffSide(obs):
    atk = mostAdvancedPlayer(obs)
    if atk == None:
        return True
    else:
        oneBehind = False
        for i in range(11):
            if obs['right_team'][i][0] > obs['left_team'][atk][0]:
                oneBehind = True
    return oneBehind
    
enemyGoal = [1, 0]

#area di tiro
perfectRange = [[0.7, 1], [-0.16, 0.16]]

#verifica se il giocatore si trova in una determinata zona del campo
def inside(pos, area):
    return area[0][0] <= pos[0] <= area[0][1] and area[1][0] <= pos[1] <= area[1][1]

@human_readable_agent
def agent(obs):
    
    goalkeeper = 0
    controlled_player_pos = obs['left_team'][obs['active']]
    
    #se il portiere ha il pallone tra i piedi : rinvia
    if controlled_player_pos == goalkeeper:
        return Action.Shot
    
    #calcio di rigore
    if obs["game_mode"] == GameMode.Penalty:
        return Action.Shot
    
    #calcio d'angolo
    if obs["game_mode"] == GameMode.Corner:
        if controlled_player_pos[0] > 0:
            return Action.HighPass
        
    #punizione
    if obs["game_mode"] == GameMode.FreeKick:
        if controlled_player_pos[0] > 0.5 and abs(controlled_player_pos[1] < 0.2):
            return Action.Shot
        else:
            return Action.LongPass
        
        
    #se il giocatore non sta correndo
    if (-0.4 < controlled_player_pos[0] < 0.3 or not (obs['right_team'][goalkeeper][0] - controlled_player_pos[0] < 0.35)) and Action.Sprint not in obs['sticky_actions']:
        return Action.Sprint
    #se il giocatore sta correndo ma è già in posizione molto avanzata
    elif (controlled_player_pos[0] > 0.6 or obs['right_team'][goalkeeper][0] - controlled_player_pos[0] < 0.35) and Action.Sprint in obs['sticky_actions']:
        return Action.ReleaseSprint
    
    
    #fase di possesso (attacco)
    if obs['ball_owned_player'] == obs['active'] and obs['ball_owned_team'] == 0:
        goalkeeper = 0
        
        inSide = not atkInOffSide(obs)
        
        #se il giocatore è in difesa e sta avanzando : lancio lungo
        if -0.6 < controlled_player_pos[0] < -0.4 and pallaAvanti(controlled_player_pos[0], obs['ball'][0]):
            return Action.HighPass
        
        #se il centravanti non è in fuorigioco: passaggio filtrante DA FIXARE
        if -0.1 < controlled_player_pos[0] < 0.1 and pallaAvanti(controlled_player_pos[0], obs['ball'][0]) and inSide:
            return Action.LongPass
            
        #se il giocatore è in una buona posizione per tirare 
        #e se la palla si trova più avanti del giocatore : tira
        elif inside(controlled_player_pos, perfectRange) and pallaAvanti(controlled_player_pos[0], obs['ball'][0]):
            return Action.Shot
        
        #se il giocatore è vicino al portiere avversario : finalizza
        elif obs['right_team'][goalkeeper][0] - controlled_player_pos[0] < 0.3: 
            
            #se il giocatore è sulla fascia : crossa
            if abs(controlled_player_pos[1]) > 0.2:
                if controlled_player_pos[1] > 0.2 and (controlled_player_pos[1] - obs['ball'][1]) > 0:
                    return Action.ShortPass
                elif controlled_player_pos[1] < -0.2 and (controlled_player_pos[1] - obs['ball'][1]) < 0:
                    return Action.ShortPass
                elif controlled_player_pos[1] > 0.2:
                    return Action.Top
                elif controlled_player_pos[1] < -0.2:
                    return Action.Bottom
                
            #se il giocatore non è sulla fascia : tira
            else:
                return Action.Shot
        
        #il giocatore corre verso la porta avversaria
        else:
            #se il giocatore è piu o meno sulla linea della porta
            if abs(controlled_player_pos[1]) < 0.10:
                return Action.Right
            #se deve correggere la traiettoria di corsa
            else:
                xdir = dirsign(enemyGoal[0] - controlled_player_pos[0])
                ydir = dirsign(enemyGoal[1] - controlled_player_pos[1])
                return directions[ydir][xdir]
        
        
        
    #fase di non possesso (difesa)
    else:
        return defensiveMove(obs)

# Play

In [ ]:
# Set up the Environment.
from kaggle_environments import make
env = make("football", configuration={"save_video": True, "scenario_name": "11_vs_11_kaggle", "running_in_notebook": True}, debug=True)
output = env.run(["/kaggle/working/submission.py", "/kaggle/working/agent.py"])[-1]
print('Left player: reward = %s, status = %s, info = %s' % (output[0]['reward'], output[0]['status'], output[0]['info']))
print('Right player: reward = %s, status = %s, info = %s' % (output[1]['reward'], output[1]['status'], output[1]['info']))
env.render(mode="human", width=800, height=600)